In [1]:
from agent.dqn.nn import GNN

/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gnn = GNN(entities=["foo", "bar"], relations=["baz", "qux"])

In [3]:
parameters = gnn.parameters()
for param in parameters:
    print(param.shape)

# Additionally, you can count the total number of parameters
total_params = sum(p.numel() for p in gnn.parameters())
print(f"Total parameters: {total_params}")

torch.Size([2, 8])
torch.Size([2, 8])
torch.Size([8])
torch.Size([8, 8])
torch.Size([8])
torch.Size([8, 8])
torch.Size([8, 8])
torch.Size([8])
torch.Size([8, 8])
torch.Size([8])
torch.Size([3, 8])
torch.Size([3])
torch.Size([8, 8])
torch.Size([8])
torch.Size([8, 8])
torch.Size([8])
torch.Size([1, 8])
torch.Size([1])
torch.Size([8, 8])
torch.Size([8])
torch.Size([8, 8])
torch.Size([8])
torch.Size([5, 8])
torch.Size([5])
torch.Size([8, 8])
torch.Size([8])
torch.Size([8, 8])
torch.Size([8])
torch.Size([1, 8])
torch.Size([1])
Total parameters: 842


In [12]:
sum(p.numel() for p in gnn.parameters())

842

In [4]:
sum(p.numel() for p in gnn.mlp_mm.parameters())

324

In [5]:
sum(p.numel() for p in gnn.mlp_explore.parameters())

342

In [6]:
sum(p.numel() for p in gnn.gnn_layers.parameters())

144

In [9]:
sum(p.numel() for p in gnn.entity_embeddings.parameters())

16

In [10]:
sum(p.numel() for p in gnn.relation_embeddings.parameters())

16

In [11]:
324 + 342 + 144 + 16 + 16

842